In [ ]:
# Code by Ian Loree and Bryce Carson
# February 12th, 2024

In [4]:
import glob
import pandas as pd
import zipfile
import os
import shutil
from pathlib import Path
from urllib.request import urlretrieve

In [5]:
def move_files(filename):
    # Remove existing directories if redownloading match files
    if Path('t20s_male_full').is_dir():
        shutil.rmtree('t20s_male_full')
    if Path('t20s_male_info').is_dir():
        shutil.rmtree('t20s_male_info')

    os.mkdir("t20s_male_full")
    os.mkdir("t20s_male_info")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('t20s_male_full')
    files = glob.glob(pathname="t20s_male_full/*info.csv")
    for file in files:
        file = file.split('/')[1]
        old_path = os.path.join("t20s_male_full", file)
        new_path = os.path.join("t20s_male_info", file)
        shutil.move(old_path, new_path)
    shutil.move("t20s_male_full/README.txt", "matches_info.txt")
    os.remove(filename)

In [6]:
# Men's T20 international matches from cricsheet.org, new format
url = "https://cricsheet.org/downloads/t20s_male_csv2.zip"
filename="t20s_male_csv2.zip"
urlretrieve(url, filename)
move_files(filename)

# Get list of all files with glob, and sort by match id
files = glob.glob(pathname="t20s_male_full/*.csv")
chrono = lambda x : int((x.split('/')[1]).split('.')[0])
files = sorted(files, key = chrono)   
dataframes = []

for file in files:
    df_to_add = pd.read_csv(file)
    # remove two columns with only one non-null entry
    df_to_add = df_to_add.drop(columns=['other_wicket_type', 'other_player_dismissed'])
    dataframes.append(df_to_add)
    
# Concatenate all the dataframes in the list into a single df
result = pd.concat(dataframes)
# produce csv file
result.to_csv('merged_files.csv')